<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Nuclear-fraction" data-toc-modified-id="Nuclear-fraction-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Nuclear fraction</a></span><ul class="toc-item"><li><span><a href="#Fitting-the-nuclear-fraction" data-toc-modified-id="Fitting-the-nuclear-fraction-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Fitting the nuclear fraction</a></span></li></ul></li><li><span><a href="#Bound-RNA-vs-size" data-toc-modified-id="Bound-RNA-vs-size-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Bound RNA vs size</a></span></li><li><span><a href="#Fitting-functions" data-toc-modified-id="Fitting-functions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Fitting functions</a></span></li></ul></li><li><span><a href="#Fitting-Haploid-data" data-toc-modified-id="Fitting-Haploid-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fitting Haploid data</a></span><ul class="toc-item"><li><span><a href="#Validating-on-diploid-data" data-toc-modified-id="Validating-on-diploid-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Validating on diploid data</a></span></li><li><span><a href="#Predicting-on-a-range-of-values-for-DNA-content" data-toc-modified-id="Predicting-on-a-range-of-values-for-DNA-content-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Predicting on a range of values for DNA content</a></span></li><li><span><a href="#Fitting-for-DNA=1.46" data-toc-modified-id="Fitting-for-DNA=1.46-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Fitting for DNA=1.46</a></span></li></ul></li><li><span><a href="#Predicting-the-single-molecule-data" data-toc-modified-id="Predicting-the-single-molecule-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Predicting the single molecule data</a></span></li><li><span><a href="#Bootstrap-fitting-on-the-parameters" data-toc-modified-id="Bootstrap-fitting-on-the-parameters-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Bootstrap fitting on the parameters</a></span></li><li><span><a href="#CIs-for-the-SMT-data" data-toc-modified-id="CIs-for-the-SMT-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>CIs for the SMT data</a></span></li><li><span><a href="#CIs-on-diploid-data" data-toc-modified-id="CIs-on-diploid-data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>CIs on diploid data</a></span><ul class="toc-item"><li><span><a href="#CIs-on-data-with-variable-DNA-content" data-toc-modified-id="CIs-on-data-with-variable-DNA-content-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>CIs on data with variable DNA content</a></span></li></ul></li><li><span><a href="#Export-files" data-toc-modified-id="Export-files-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Export files</a></span></li></ul></div>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

from scipy.optimize import curve_fit

import toml

In [ ]:
col_vol = 'cell_volume_fL'
col_RNA = 'Rpb1_occupancy_ChIP-seq'

In [ ]:
with open('../config.toml', 'r') as f:
    config = toml.load(f)

In [ ]:
PATH = config["DATADIR"]

# Description

This notebook contains the main analysis of the deterministic Dynamic Equilibrium model, including comparison with ChIP-seq data. 



It requires having the data in a `data_in` sub-directory in the `DATADIR`.

# Load data

## Nuclear fraction

In [ ]:
vols = pd.read_csv(
    os.path.join(
    PATH, "data_in", "230101_nuc_vol_F0-F7_updated.txt"), delimiter="\t"
)

In [ ]:
vols['nuc_frac'] = vols['nucleus_vol_um3'] / vols['cell_vol_um3']

In [ ]:
plt.scatter(vols['cell_vol_um3'], vols['nuc_frac'])

### Fitting the nuclear fraction

In [ ]:
offset=20

In [ ]:
def nuc_frac_model(x, alpha, beta, delta): 
    return alpha + beta * np.exp(-delta * (x-offset))

In [ ]:
# fit curve
(alpha, beta, delta), _ = curve_fit(nuc_frac_model, vols['cell_vol_um3'], vols['nuc_frac'])

Parameters of the fit

In [ ]:
alpha

In [ ]:
beta

In [ ]:
delta

In [ ]:
xs = np.linspace(10, 200, 100)

plt.scatter(vols['cell_vol_um3'], vols['nuc_frac'])
plt.plot(xs, nuc_frac_model(xs, alpha, beta, delta), ls = '--', lw=4, c='red')
plt.xlabel("Cell Volume [um3]")
plt.ylabel("Nuclear Fraction [-]")

## Bound RNA vs size

Load data

In [ ]:
df_ChIP = pd.read_csv(
    os.path.join(PATH, "data_in", '221108_ChIPdata_summary.txt'), 
    delimiter='\t').dropna(axis=1)
df_ChIP = df_ChIP.rename(columns={c: c.strip() for c in df_ChIP.columns})

In [ ]:
df_SMT = pd.read_csv(
    os.path.join(PATH, "data_in", '221108_SMTdata_summary.txt'), 
    delimiter='\t'
).dropna(axis=1)

In [ ]:
df_146 = pd.read_csv(
    os.path.join(PATH, "data_in", '230122_cell_size_mutant_ChIPdata_summary.txt'), 
    delimiter='\t', usecols=[1, 2]
).dropna(axis=0)

In [ ]:
df_146.rename(
    columns={df_146.columns[0]: col_vol, 
    df_146.columns[1]: col_RNA}, 
    inplace=True
)

In [ ]:
df_146

## Fitting functions

In [ ]:
def nuc_vol_lin(x):
    return  0.038 * x + 0.55

In [ ]:
def obj1(x, a, b): 
    """Initial model with constant nuclear fraction"""
    return a*(x/b)/(1+x*b)

def obj2(x, a, b): 
    """Model with nuclear fraction decreasing as 1/x"""
    nuclear_volume = nuc_vol_lin(x)
    return a * (x/b) / (1 + b * nuclear_volume)

def obj3(x, a, b):
    """Model with nuclear fraction decreasing as exp(-delta*x)"""
    nuc_volume = x * (alpha + beta * np.exp(-delta * (x-offset)))
    return a * (x/b) / (1 + b * nuc_volume)

In [ ]:
models = ['Constant nuclear fraction', 'Nuclear fraction as 1/x', 'Nuclear fraction as exp(-delta*x)']

# Fitting Haploid data

In [ ]:
df_hap = df_ChIP.loc[df_ChIP['strain']=="MS64"]

In [ ]:
# fit curve
(a1, b1), _ = curve_fit(obj1, df_hap[col_vol], df_hap[col_RNA])

(a2, b2), _ = curve_fit(obj2, df_hap[col_vol], df_hap[col_RNA])

(a3, b3), _ = curve_fit(obj3, df_hap[col_vol], df_hap[col_RNA])

params_fit=[(a1, b1), (a2, b2), (a3, b3)]

In [ ]:
# define new input values
x_new = np.linspace(1,230,300)

# use optimal parameters to calculate new values
y1 = obj1(x_new, a1, b1)
y2 = obj2(x_new, a2, b2)
y3 = obj3(x_new, a3, b3)

ys = [y1, y2, y3]

In [ ]:
fig = plt.figure()
plt.plot(df_hap[col_vol], df_hap[col_RNA], 'ro', label='Replicates')
plt.plot(x_new, y1, 'r')
plt.plot(x_new, y2, 'g--')
plt.plot(x_new, y3, 'b--')

plt.xlabel('cell size (fl)')
plt.ylabel('DNA bound RNA Pol II')

plt.legend()
plt.grid()

## Validating on diploid data

In [ ]:
df_dip = df_ChIP.loc[df_ChIP['strain']=='MS67']

In [ ]:
fig = plt.figure()
plt.scatter(df_dip[col_vol], df_dip[col_RNA])

y1_dip = obj1(x_new, a1/2, b1/2)
y2_dip = obj2(x_new, a2/2, b2/2)
y3_dip = obj3(x_new, a3/2, b3/2)

ys_dip = [y1_dip, y2_dip, y3_dip]

plt.plot(x_new, y1_dip)
plt.plot(x_new, y2_dip)
plt.plot(x_new, y3_dip)

## Predicting on a range of values for DNA content

In [ ]:
DNA_content = [1.2, 1.4, 1.46, 1.6, 1.8,]
# DNA_content=[1.46]

fig = plt.figure()
for DNAc in DNA_content:
    plt.plot(x_new, obj3(x_new, a3/DNAc, b3/DNAc), label=f"DNA = {DNAc}")
plt.xlabel("cell size")
plt.ylabel("Occ")
plt.legend()

## Fitting for DNA=1.46

In [ ]:
df_146

In [ ]:
(a146, b146), _ = curve_fit(obj3, df_146[col_vol], df_146[col_RNA])

In [ ]:
y146 = obj3(x_new, a146, b146)

y1_146 = obj3(x_new, a146 * 1.46, b146 * 1.46)
y2_146 = obj3(x_new, a146 * 1.46/2, b146 * 1.46/2)

In [ ]:
plt.scatter(df_146[col_vol], df_146[col_RNA])
plt.plot(x_new, y146, "--", label="fit DNA=1.46")

plt.plot(x_new, y1_146, label="pred DNA=1")
plt.plot(x_new, y2_146, label="pred DNA=2")

# plt.scatter(df_dip[col_vol], df_dip[col_RNA])
# plt.scatter(df_hap[col_vol], df_hap[col_RNA])

plt.legend()
plt.grid()
plt.xlabel("Cell size [fL]")
plt.ylabel("Occ")

# Predicting the single molecule data

In [ ]:
col_RNA_frac = 'Rpb1_bound_fraction_mean'

In [ ]:
# setting the axes at the centre
fig = plt.figure()

# plot the function
plt.plot(df_SMT[col_vol], df_SMT[col_RNA_frac], 'go', label="data")

err1 = np.sqrt(np.mean((obj1(df_SMT[col_vol], a1, b1)/(a1*df_SMT[col_vol]/b1) - df_SMT[col_RNA_frac])**2))
plt.plot(x_new, y1/(a1*x_new/b1), 'r', label=f"constant nuclear fraction, err={np.around(err1, decimals=3)}")

err2 = np.sqrt(np.mean((obj2(df_SMT[col_vol], a2, b2)/(a2*df_SMT[col_vol]/b2) - df_SMT[col_RNA_frac])**2))
plt.plot(x_new, y2/(a2*x_new/b2), label=f"1/x nuclear fraction, err={np.around(err2, decimals=3)}")

err3 = np.sqrt(np.mean((obj3(df_SMT[col_vol], a3, b3)/(a3*df_SMT[col_vol]/b3) - df_SMT[col_RNA_frac])**2))
plt.plot(x_new, y3/(a3*x_new/b3), label=f"exponential nuclear fraction, err={np.around(err3, decimals=3)}")

plt.legend()
plt.grid()
plt.xlabel('cell size (fl)')
plt.ylabel('RNAP II bound fraction')
plt.xlim([15, 175])


# Bootstrap fitting on the parameters

In [ ]:
bag_size = 7
n_fits = 100

params = {1: {'a': [], 'b': []}, 2: {'a': [], 'b': []}, 3: {'a': [], 'b': []}}
preds = {1: [], 2: [], 3: []}
SMT_preds = {1: [], 2: [], 3: []}
DIP_preds = {1: [], 2: [], 3: []}

objs = [obj1, obj2, obj3]

for _ in range(n_fits):
    
    idx = np.random.choice(len(df_hap), bag_size)
    
    x_crt = df_hap[col_vol].iloc[idx].values
    y_crt = df_hap[col_RNA].iloc[idx].values
    
    for (k, obj) in enumerate(objs):
        (a, b), _ = curve_fit(obj, x_crt, y_crt)
        params[k+1]['a'].append(a)
        params[k+1]['b'].append(b)
        y = obj(x_new, a, b)
        preds[k+1].append(y.reshape(-1, 1))
        SMT_preds[k+1].append((y/(a * x_new/b)).reshape(-1, 1))
        DIP_preds[k+1].append((obj(x_new, a/2, b/2)).reshape(-1, 1))

In [ ]:
# plotting the quantiles of the predictions
quantiles = [.1, .5, .9]

_, ax = plt.subplots(1, 3, figsize=(15, 4))

for k in range(3):
    Qs = [np.quantile(np.concatenate(preds[k+1], axis=1), q, axis=1) for q in quantiles]
    
    for Q in Qs: 
        ax[k].plot(x_new, Q, color='gray')
    ax[k].scatter(df_hap[col_vol], df_hap[col_RNA])
    
    ax[k].set_title(f"Model {k+1}")
    ax[k].grid()

# CIs for the SMT data

In [ ]:
# quantiles of the SMT predictions

SMT_Qs = {1: [], 2: [], 3: []}
quantiles_SMT = [.1, .9]

for k in range(3):
    SMT_Qs[k+1] = [np.quantile(np.concatenate(SMT_preds[k+1], axis=1), q, axis=1) for q in quantiles_SMT]

In [ ]:
COLORS = ['firebrick', 'forestgreen', 'darkblue']
alpha_plot=.2

In [ ]:
for k in range(3):
    

    plt.fill_between(x_new, SMT_Qs[k+1][0], SMT_Qs[k+1][1], 
                     color = COLORS[k], alpha=alpha_plot
                    )
    plt.plot(
        x_new, ys[k]/(params_fit[k][0]*x_new/params_fit[k][1]), c=COLORS[k], 
        label="constant nuclear fraction"
    )
        
        
# plot the function
plt.scatter(df_SMT[col_vol], df_SMT[col_RNA_frac], color='black', label="data")

plt.legend()
plt.grid()
plt.xlabel('cell size (fl)')
plt.ylabel('RNAP II bound fraction')
plt.xlim([15, 175])

In [ ]:
_, ax = plt.subplots(1, 3, figsize=(15, 4))

for k in range(3):
    
    ax[k].fill_between(x_new, SMT_Qs[k+1][0], SMT_Qs[k+1][1], 
                     color = COLORS[k], alpha=alpha_plot
                    )
    ax[k].plot(
        x_new, ys[k]/(params_fit[k][0]*x_new/params_fit[k][1]), c=COLORS[k], 
        label="constant nuclear fraction"
    )
        
        
    # plot the function
    ax[k].scatter(df_SMT[col_vol], df_SMT[col_RNA_frac], color='black', label="data")

#     plt.legend()
    ax[k].grid()
    ax[k].set_xlabel('cell size (fl)')
    ax[k].set_ylabel('RNAP II bound fraction')
    ax[k].set_xlim([15, 175])
    ax[k].set_ylim([0, 1])
    ax[k].set_title(models[k])


# CIs on diploid data

In [ ]:
# quantiles of the SMT predictions

DIP_Qs = {1: [], 2: [], 3: []}
quantiles_DIP = [.1, .9]

for k in range(3):
    DIP_Qs[k+1] = [np.quantile(np.concatenate(DIP_preds[k+1], axis=1), q, axis=1) for q in quantiles_DIP]

In [ ]:
_, ax = plt.subplots(1, 3, figsize=(15, 4))

for k in range(3):
    
    ax[k].fill_between(x_new, DIP_Qs[k+1][0], DIP_Qs[k+1][1], 
                     color = COLORS[k], alpha=alpha_plot
                    )
    ax[k].plot(
        x_new, ys_dip[k], c=COLORS[k], 
        label="constant nuclear fraction"
    )
        
        
    # plot the function
    ax[k].scatter(df_dip[col_vol], df_dip[col_RNA], color='black', label="data")

#     plt.legend()
    ax[k].grid()
    ax[k].set_xlabel('cell size (fl)')
    ax[k].set_ylabel('RNAP II bound fraction')
#     ax[k].set_xlim([15, 175])
#     ax[k].set_ylim([0, 1])
    ax[k].set_title(models[k])


## CIs on data with variable DNA content

In [ ]:
var_DNA_preds = {}
var_DNA_qs = {}
quantiles_var_DNA = [.1, .5, .9]

for DNAc in DNA_content:
    var_DNA_preds[DNAc] = {1: [], 2: [], 3: []}
    var_DNA_qs[DNAc] = {1: [], 2: [], 3: []}
    
    for (k, obj) in enumerate(objs): #iterate over the model
        for (a, b) in zip(params[k+1]['a'], params[k+1]['b']):
            var_DNA_preds[DNAc][k+1].append((obj(x_new, a/DNAc, b/DNAc)).reshape(-1, 1))
            
        var_DNA_qs[DNAc][k+1] = [np.quantile(np.concatenate(var_DNA_preds[DNAc][k+1], axis=1), q, axis=1) for q in quantiles_var_DNA]

In [ ]:
fig = plt.figure()

for DNAc in DNA_content:
    plt.plot(x_new, var_DNA_qs[DNAc][3][1], label=f"DNA = {DNAc}")
    plt.fill_between(x_new, var_DNA_qs[DNAc][3][0], var_DNA_qs[DNAc][3][2], alpha=.2)
    
plt.legend()
plt.xlabel("cell size")
plt.ylabel("Rpb1")
plt.grid()

# Export files

Files are exported in the `data_out` subdirectory in `DATADIR`.

In [ ]:
model_names = ['constant_nuclear_fraction', 'linear_nuclear_fraction', 'exp_nuclear_fraction']

In [ ]:
for (k, model) in enumerate(model_names): 
    df_final = pd.DataFrame()
    df_final.index = x_new
    df_final.index.rename('cell_volume_fL', inplace=True)
    
    df_final['Rpb1_occupancy_haploid_fit'] = ys[k]
    df_final['Rpb1_occupancy_haploid_fit_10pc'] = 0
    df_final['Rpb1_occupancy_haploid_fit_90pc'] = 0
    
    df_final['Rpb1_occupancy_diploid_prediction'] = ys_dip[k]
    df_final['Rpb1_occupancy_diploid_prediction_10pc'] = DIP_Qs[k+1][0]
    df_final['Rpb1_occupancy_diploid_prediction_90pc'] = DIP_Qs[k+1][1]
    
    for DNAc in DNA_content:
        
        df_final[f"Rpb1_occupancy_DNA{DNAc}_prediction_10pc"] = var_DNA_qs[DNAc][k+1][0]
        df_final[f"Rpb1_occupancy_DNA{DNAc}_prediction_50pc"] = var_DNA_qs[DNAc][k+1][1]
        df_final[f"Rpb1_occupancy_DNA{DNAc}_prediction_90pc"] = var_DNA_qs[DNAc][k+1][2]
    
    df_final['Rpb1_bound_fraction_haploid_prediction'] = ys[k]/(params_fit[k][0] * x_new/params_fit[k][1])
    df_final['Rpb1_bound_fraction_haploid_prediction_10pc'] = SMT_Qs[k+1][0]
    df_final['Rpb1_bound_fraction_haploid_prediction_90pc'] = SMT_Qs[k+1][1]
    
    if model=="exp_nuclear_fraction":
        df_final['Rpb1_occupancy_DNA1.46_fit'] = y146
        df_final['Rpb1_occupancy_DNA1.46_prediction_haploid'] = y1_146
        df_final['Rpb1_occupancy_DNA1.46_prediction_diploid'] = y2_146
    
    
    if k==0:
        nuc_vol = "?"
        nuc_frac = "C"
    elif k==1:
        nuc_vol = nuc_vol_lin(x_new)
        nuc_frac = nuc_vol/x_new
    elif k==2:
        nuc_vol = nuc_frac_model(x_new, alpha, beta, delta) * x_new
        nuc_frac = nuc_vol/x_new
        
    df_final['nuclear_fraction'] = nuc_frac
    df_final['nuclear_volume'] = nuc_vol
    
    df_final.to_csv(
        os.path.join(PATH, "data_out", f"data_{model}.csv")
    )